In [106]:
import pandas as pd
import numpy as np
import pyspark.sql.functions as f

def get_spark_env(version, job_name, n):
    if version in ["2.4"]:
        from pyspark.sql import SparkSession
        spark = SparkSession.builder.master("local[*]").config('spark.executor.memoryOverhead.config', n).appName(job_name).getOrCreate()
        return spark
    elif version in ["1.6"]:
        return None, None

def Print_DF(DF):
    return DF.select("*").toPandas()

def Print_DF_Rows(DF,NbRows):
    return pd.DataFrame(DF.take(NbRows),columns=DF.columns)

job_name = 'Formation'
spark = get_spark_env("2.4", job_name, '3')
print (spark.version)

3.1.1


In [144]:
def delColumn(df, drop_list):
    """
    Fonction qui supprimer les colonne comportant le nom donné

    df: pandas.dataframe -> data a traité
    drop_list: str[] -> valeur qui permettras d'effectuer la suppression des colonne choisi
    """
    
    return df.select([column for column in df.columns if column not in drop_list])

def cleanerData(df, column=[], letNan=False):
    """
    Fonction qui a pour role d'éffacer les valeurs erroné dans la dataframe
    
    df: pandas.dataframe -> data a traité
    column: str[] -> selectionne la colonne en particulier
    letNan: bool -> si il est activé toute les valeurs erronés seront null et non remplacé par la moyenne de la colonne
    """
    
    if column == []:
        for col in df.columns:
            df.withColumn(col, when(type(f.col(col)) == str and f.col(col) < 0,np.nan))
    else:
        pass
    return df



def decribeData(df, nbNan=False, selectColumns=(), mode=""):
    """
    Fonction pour generer une description de la data selon differente option
    
    df: pandas.dataframe -> data a traité
    nbNan: bool -> pour compter le nombre de valeur null
    selectColumns: str() -> pour traiter des colonne specifique
    mode: str -> mode possible; mean, min, max, median, type:int[] liste des quantiles recherché
    """
    described = df
    if selectColumns != ():
        described = df.select(*selectColumns)
    
    if nbNan:
        if selectColumns != ():
            print('\033[93m' + "SELECTCOLUMNS Warn:"+ '\033[95m' +" si nbNan est actif SELECTCOLUMNS ne sert a rien")
        if mode != "":
            print('\033[93m' + "MODE Warn:"+ '\033[95m' +" si nbNan est actif MODE ne sert a rien")
        if roundValue != -1:
            print('\033[93m' + "ROUNDVALUE Warn:"+ '\033[95m' +" si nbNan est actif ROUNDVALUE ne sert a rien")
        return df.select([f.count(f.when(f.col(c).isNull(), c)).alias(c) for c in df.columns])
    
    if mode == "mean":
        described = df.select(*[f.avg(c) for c in df.columns])
    elif mode == "min":
        described =  df.select(*[f.min(c).alias(c) for c in df.columns])
    elif mode == "max":
        described =  df.select(*[f.max(c).alias(c) for c in df.columns])
    elif mode == "median":
        described =  df.approxQuantile([c for c in df.columns],[0.5], 0.25)
    elif type(mode) == list:
        return df.approxQuantile([c for c in df.columns],mode, 0.25)

    if mode != "":
        return described
    
    described = described.summary()
    
    return described

In [142]:
#VERSION PANDAS
def cleanerData(df, columns=[], letNan=False):
    """
    Fonction qui a pour role d'éffacer les valeurs erroné dans la dataframe
    
    df: pandas.dataframe -> data a traité
    column: str[] -> selectionne la colonne en particulier
    letNan: bool -> si il est activé toute les valeurs erronés seront null et non remplacé par la moyenne de la colonne
    """
    if columns == []:
        for col in df:
            df[col] = pd.to_numeric(df[col],errors='coerce')
            df[col] = df[col].apply(lambda x: np.nan if x < 0 else x)
            if not letNan :
                mean = df[col].mean()
                df[col] = df[col].apply(lambda x: mean if x == 0 or np.isnan(x) else x)
    else:
        for col in columns:
            df[col] = pd.to_numeric(df[col],errors='coerce')
            df[col] = df[col].apply(lambda x: np.nan if x < 0 else x)
            if not letNan:
                mean = df[col].mean()
                df[col] = df[col].apply(lambda x: mean if x == 0 or np.isnan(x) else x)
    return df



In [143]:
df = delColumn(df, ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1', '_c0'])
#tmp = cleanerData(df.toPandas())
#df = spark.createDataFrame(tmp)
df = cleanerData(df)
df.toPandas()

AnalysisException: filter expression '`fixed acidity`' of type string is not a boolean.;
Filter fixed acidity#4314: string
+- Project [fixed acidity#4314, volatile acidity#4315, citric acid#4316, residual sugar#4317, chlorides#4318, free sulfur dioxide#4319, total sulfur dioxide#4320, density#4321, pH#4322, sulphates#4323, alcohol#4324, quality#4325]
   +- Project [fixed acidity#4314, volatile acidity#4315, citric acid#4316, residual sugar#4317, chlorides#4318, free sulfur dioxide#4319, total sulfur dioxide#4320, density#4321, pH#4322, sulphates#4323, alcohol#4324, quality#4325]
      +- Project [fixed acidity#4314, volatile acidity#4315, citric acid#4316, residual sugar#4317, chlorides#4318, free sulfur dioxide#4319, total sulfur dioxide#4320, density#4321, pH#4322, sulphates#4323, alcohol#4324, quality#4325]
         +- Project [fixed acidity#4314, volatile acidity#4315, citric acid#4316, residual sugar#4317, chlorides#4318, free sulfur dioxide#4319, total sulfur dioxide#4320, densit

In [124]:
df = spark.read.load('Dataset_6_quality.csv', format="csv", sep="|", inferSchema="true", header="true")

In [125]:
decribeData(df, selectColumns=("residual sugar", "citric acid", "volatile acidity"), mode="median")

AnalysisException: Can't extract value from Unnamed: 0#4308: need struct type but got int